## Quiz #0502

### "Human Activity Recognition" 

#### Answer the following questions by providing Python code:
#### Objectives:
- Carry out the EDA.
- Carry out the data pre-processing.
- Optimize and test a predictive model of your choice.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn import metrics, preprocessing
warnings.filterwarnings(action='ignore')                  # Turn off the warnings.
%matplotlib inline

#### Read in data:
The explanation on the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones).

In [2]:
# Go to the directory where the data file is located. 
# os.chdir(r'~~')                # Please, replace the path with your own. 

In [3]:
df = pd.read_csv('data_human activity recognition.csv', header='infer')

In [4]:
df.shape

(19622, 160)

In [5]:
print(df.columns)

Index(['Unnamed: 0', 'user_name', 'raw_timestamp_part_1',
       'raw_timestamp_part_2', 'cvtd_timestamp', 'new_window', 'num_window',
       'roll_belt', 'pitch_belt', 'yaw_belt',
       ...
       'gyros_forearm_x', 'gyros_forearm_y', 'gyros_forearm_z',
       'accel_forearm_x', 'accel_forearm_y', 'accel_forearm_z',
       'magnet_forearm_x', 'magnet_forearm_y', 'magnet_forearm_z', 'classe'],
      dtype='object', length=160)


1). Carry out the EDA. Check for the missing values. HINT: The response variable is 'classe'.

In [6]:
df.isnull().sum()

Unnamed: 0              0
user_name               0
raw_timestamp_part_1    0
raw_timestamp_part_2    0
cvtd_timestamp          0
                       ..
accel_forearm_z         0
magnet_forearm_x        0
magnet_forearm_y        0
magnet_forearm_z        0
classe                  0
Length: 160, dtype: int64

2). Get rid of the columns that have more than 97% missing values.

In [7]:
for column in df.columns:
    if df[column].isnull().sum()/df.shape[0] > 0.97 :
        df.drop(columns=column,inplace=True)

3). Get rid of the unnecessary columns. HINT: Those columns with "time" in the name and those that are obviously unnecessary.

In [8]:
df.drop(columns=['user_name','num_window','Unnamed: 0'],inplace=True)
for column in df.columns:
    if "time" in column :
        df.drop(columns=column,inplace=True)

4). Label encode the responde variable. HINT: use preprocessing.LabelEncoder(). 

In [9]:
df["new_window"]=df["new_window"].map({'yes': 1, 'no': 0})
le = preprocessing.LabelEncoder()
le.fit(df["classe"])
df["classe"]=le.transform(df["classe"])

In [10]:
df.values

array([[   0.  ,    1.41,    8.07, ...,  654.  ,  476.  ,    0.  ],
       [   0.  ,    1.41,    8.07, ...,  661.  ,  473.  ,    0.  ],
       [   0.  ,    1.42,    8.07, ...,  658.  ,  469.  ,    0.  ],
       ...,
       [   0.  ,  145.  ,  -35.5 , ..., -654.  ,  -70.  ,    4.  ],
       [   0.  ,  143.  ,  -35.9 , ..., -678.  ,  -98.  ,    4.  ],
       [   1.  ,  143.  ,  -36.  , ..., -686.  , -110.  ,    4.  ]])

5). Carry out min-max scaling of the exploratory variables. HINT: use preprocessing.MinMaxScaler().

In [11]:
target= df.classe
x = df.drop(columns=['classe']).values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.insert(53,'classe',target)
df.rename(columns={0:'new_window', 1:'roll_belt', 2:'pitch_belt', 3:'yaw_belt', 4:'total_accel_belt',
       5:'gyros_belt_x', 6:'gyros_belt_y', 7:'gyros_belt_z', 8:'accel_belt_x',
       9:'accel_belt_y', 10:'accel_belt_z', 11:'magnet_belt_x', 12:'magnet_belt_y',
       13:'magnet_belt_z', 14:'roll_arm', 15:'pitch_arm', 16:'yaw_arm', 17:'total_accel_arm',
       18:'gyros_arm_x', 19:'gyros_arm_y', 20:'gyros_arm_z', 21:'accel_arm_x',
       22:'accel_arm_y', 23:'accel_arm_z', 24:'magnet_arm_x', 25:'magnet_arm_y',
       26:'magnet_arm_z', 27:'roll_dumbbell', 28:'pitch_dumbbell', 29:'yaw_dumbbell',
       30:'total_accel_dumbbell', 31:'gyros_dumbbell_x', 32:'gyros_dumbbell_y',
       33:'gyros_dumbbell_z', 34:'accel_dumbbell_x',35: 'accel_dumbbell_y',
       36:'accel_dumbbell_z', 37:'magnet_dumbbell_x', 38:'magnet_dumbbell_y',
       39:'magnet_dumbbell_z', 40:'roll_forearm', 41:'pitch_forearm', 42:'yaw_forearm',
       43:'total_accel_forearm', 44:'gyros_forearm_x', 45:'gyros_forearm_y',
       46:'gyros_forearm_z', 47:'accel_forearm_x', 48:'accel_forearm_y',
       49:'accel_forearm_z', 50:'magnet_forearm_x', 51:'magnet_forearm_y',
       52:'magnet_forearm_z'},inplace=True)
df.head()

,new_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,gyros_belt_x,gyros_belt_y,gyros_belt_z,accel_belt_x,accel_belt_y,...,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,classe
0,0.0,0.158774,0.550129,0.23844,0.103448,0.319018,0.500000,0.467532,0.482927,0.313305,...,0.848286,0.022074,0.033753,0.707692,0.536977,0.313433,0.647029,0.652357,0.702375,0
1,0.0,0.158774,0.550129,0.23844,0.103448,0.325153,0.500000,0.467532,0.478049,0.313305,...,0.847901,0.022074,0.033753,0.707692,0.536977,0.312076,0.646516,0.655303,0.700921,0
2,0.0,0.158827,0.550129,0.23844,0.103448,0.319018,0.500000,0.467532,0.487805,0.317597,...,0.848286,0.022011,0.033837,0.711795,0.537621,0.316147,0.646516,0.654040,0.698982,0
3,0.0,0.159141,0.549957,0.23844,0.103448,0.325153,0.500000,0.464286,0.478049,0.309013,...,0.847901,0.022011,0.033837,0.704615,0.538907,0.314790,0.647541,0.654040,0.698982,0
4,0.0,0.159141,0.550129,0.23844,0.103448,0.325153,0.515625,0.467532,0.482927,0.304721,...,0.847901,0.022074,0.033753,0.704615,0.538907,0.314790,0.647029,0.652778,0.700921,0


6). Choose an algorithm and carry out the predictive analysis.

- Optimize the hyperparameter(s)
- Calculate the accuracy.
- Aim for upper 90% accuracy.

In [12]:
x=df.drop(columns=['classe'])
y=df.classe

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2) #splitting data with test size of 20%

accuracies=[0]
for n_estimators in range(1,100):
    model=RandomForestClassifier(n_estimators)  #Creating a random forest with 100 decision trees
    model.fit(x_train, y_train)  #Training our model
    y_pred=model.predict(x_test)  #testing our model
    if metrics.accuracy_score(y_test, y_pred)> max(accuracies):
        best=n_estimators
    accuracies.append(metrics.accuracy_score(y_test, y_pred))
print("Number of estimators:",best)
print("Accuracy:", max(accuracies))

Number of estimators: 56
Accuracy: 0.9964331210191083


# Warning
### It took about 5min to train and find the best accuracy, I advise you to reduce the interval range(1,100) to range(1,10) if you want to test it.

Number of estimators: 56

Accuracy: 0.9961783439490446